# Keras model 을 opencv 에서 써보기
- keras 에서 저장된 모델(hd5)을 opencv에서 이해할 수 없으므로 따로 pb 파일로 변환을 해주어야 함

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K #이건 뭐하는 모듈? 

# from tensorflow.compat.v1.keras import backend as K
# import tensorflow.compat.v1 as tf #호환성을 위해 별도로 만들어진 패키지, 하위버전
# tf.disable_v2_behavior()

Using TensorFlow backend.


In [2]:

X = np.array([[0,0], [0,1], [1,0], [1,1]], 'float32')
Y = np.array([[0], [1], [1], [0]], 'float32')

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, input_dim=2, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation ='sigmoid', name='Output'))


model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])

model.fit(X, Y, batch_size=1, epochs=100, verbose=0)


model.summary()

#input.op.name 텐서플로우가 가지고 있는 실제 name

# inputs:  ['dense_input']
print('inputs: ', [input.op.name for input in model.inputs])

# outputs:  ['dense_4/Sigmoid']
print('outputs: ', [output.op.name for output in model.outputs])

#model save
model.save('xor.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
Output (Dense)               (None, 1)                 65        
Total params: 12,737
Trainable params: 12,737
Non-trainable params: 0
_________________________________________________________________
inputs:  ['dense_input']
outputs:  ['Output/Identity']


In [ ]:
model.predict(np.array([[1, 1], [0,1]]))

In [ ]:
# from tensorflow.compat.v1.keras import backend as K
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()


def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph


#K.get_session() 열려있는 세션을 가져옴
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs]) #'output/Sigmoid'
tf.train.write_graph(frozen_graph, './', 'xor.pbtxt', as_text=True) #txt로 저장
tf.train.write_graph(frozen_graph, './', 'xor.pb', as_text=False) #바이너리로 저장

In [ ]:
import cv2 as cv2
net = cv2.dnn.readNetFromTensorflow('xor.pb')
layersNames = net.getLayerNames()
print(layersNames)

In [ ]:
net.setInput(np.array([[0, 0], [0,1], [1,0], [1,1]]))
out = net.forward(outputName='Output/Sigmoid')
print(out)

out = net.forward()
print(out)